In [1]:
import pandas as pd
import requests
import zipfile
import numpy as np
from datetime import datetime, timedelta
import os
import shutil
import sys
from zipfile import BadZipFile
from multiprocessing import Pool
import time
import io
from sqlalchemy import create_engine, inspect
import psycopg2
import logging
import requests
import psutil
import logging
from threading import Lock
from concurrent.futures import ThreadPoolExecutor
import csv
from io import StringIO
import asyncio
import aiofiles

sys.setrecursionlimit(10000)
#curdir = os.path.abspath(os.getcwd())
#if not curdir.endswith(os.sep):
#    curdir += os.sep
# http://data.gdeltproject.org/gdeltv2/masterfilelist.txt

curdir="/Volumes/T5EVO/gdelt_download/"

D:\Software\anaconda\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
D:\Software\anaconda\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
D:\Software\anaconda\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
D:\Software\anaconda\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
# Functions
def process_url(url):
    try:
        # Split to get the filename
        name = url.split("/")[-1]
        names = name.split(".")

        # Download the file
        download_url(url, curdir + name)

        # Unzip the file
        with zipfile.ZipFile(curdir + name, 'r') as zip_ref:
            zip_ref.extractall(curdir + names[0])

        # Read the CSV file and process data
        news = pd.read_csv(curdir + 
                           names[0] + '/' + 
                           names[0] + "." + 
                           names[1] + "." + 
                           names[2], 
                           on_bad_lines='skip', 
                           delimiter="\t", names=["GKGRECORDID", "DATE", 
                                                  "SourceCollectionIdentifier", "SourceCommonName", 
                                                  "DocumentIdentifier", "Counts", "V2Counts", 
                                                  "Themes", "V2Themes", "Locations", "V2Locations", 
                                                  "Persons", "V2Persons", "Organizations", "V2Organizations", 
                                                  "V2Tone", "Dates", "GCAM", "SharingImage", "RelatedImages", 
                                                  "SocialImageEmbeds", "SocialVideoEmbeds", "Quotations", 
                                                  "AllNames", "Amounts", "TranslationInfo", "Extras"], 
                                                  dtype="string", 
                                                  encoding = "ISO-8859-1")
        # "GKGRECORDID", "DATE", 
        # "SourceCollectionIdentifier", "SourceCommonName", 
        # "DocumentIdentifier", "Counts", "V2Counts", 
        # "Themes", "V2Themes", "Locations", "V2Locations", 
        # "Persons", "V2Persons", "Organizations", "V2Organizations", 
        # "V2Tone", "Dates", "GCAM", "SharingImage", "RelatedImages", 
        # "SocialImageEmbeds", "SocialVideoEmbeds", "Quotations", 
        # "AllNames", "Amounts", "TranslationInfo", "Extras"

        # Clean up the downloaded and extracted files
        #shutil.move(curdir + name, f'{curdir}zip_2024/{name}') # save the zip to target file instead of deleting
        os.remove(curdir + name)
        shutil.rmtree(curdir + names[0])

        return news, name, names

    except Exception as e:
        # print(f"Error processing {url}: {e}")
        # Write the URL to a text file
        with open('error_urls.txt', 'a') as f:
            f.write(f"{url}\n")
        return False  # Indicate failure
    
def download_url(url, save_path, chunk_size=128):
    r = requests.get(url, stream=True)
    with open(save_path, 'wb') as fd:
        for chunk in r.iter_content(chunk_size=chunk_size):
            fd.write(chunk)

def generate_gdelt_urls(start_year, start_month, start_day, end_year, end_month, end_day):
    start_date = datetime(start_year, start_month, start_day)
    current_date = datetime(end_year, end_month, end_day) # change to the end of the year

    delta = current_date - start_date
    total_intervals = int(delta.total_seconds() / 900)

    urls = []

    for i in range(total_intervals):
        interval_date = start_date + timedelta(minutes = 15 * i)

        formatted_date = interval_date.strftime('%Y%m%d%H%M%S')

        url = f'http://data.gdeltproject.org/gdeltv2/{formatted_date}.gkg.csv.zip'
        urls.append(url)

    return urls


def clean_bot(x):
    """
    The actual robot which is gonna clean the company suffix for every company.
    :type x: str
    :rtype: str
    """
    if len(x) > 0 and x[-1] in redundant:
        del x[-1]
        clean_bot(x)
    return x

def clean(x):
    """
    Clean all the organizations from the 'Organizations' column in the raw data file.
    The data type of every entry in the 'Organizations' column is string.
    :type x: str
    :rtype: list
    """
    x = x.split(';')
    temp = list()
    for y in x:
        y = y.split(' ')
        temp.append(clean_bot(y))
    return temp

def del_countries(x):
    """
    Delete the strings which are names of countries in every entry from the 'Organizations' column.
    :type: list (list of lists)
    :rtype: list (list of lists)
    """
    x = [k for k in x if ' '.join(k).lower() not in nations]
    return x

def find_index(corp_name, middle_index, company_key1):
    """
    Precisely find the index of the objective string (company name) by setting ',' as delimiter and loop the string.
    Return the index if found, return -1 if not found.
    :type: string, int, string
    :rtype: int
    """
    middle2 = company_key1[middle_index:].find(corp_name)
    if middle2 == -1:
        return middle2
    elif middle2 == 0:
        return middle2
    elif company_key1[middle2 + middle_index - 1] == ',':
        return middle2 + middle_index
    else:
        index1 = company_key1[middle2 + middle_index:].find(',')
        if index1 == -1:
            return -1
        count1 = middle2 + middle_index + index1
        return find_index(corp_name, count1, company_key1)

def company_matching(x, company_key1, company_dict):
    """
    Match the objective string (company name) and return a list of found companies.
    :type: list (list of lists)
    :type company_key1: string
    :type company_dict: dictionary
    :rtype: list
    """
    x1 = [' '.join(y) for y in x]
    childs = list()
    for i in x1:
        head_index = find_index(i, 0, company_key1)

        if head_index == -1:
            continue
        else:
            if company_key1[head_index:].find(',') == -1:
                childs.append(company_key1[head_index:])
            else:
                childs.append(company_key1[head_index:head_index + company_key1[head_index:].find(',')])

    if len(childs) == 0:
        return np.nan

    else:
        return childs

def return_first_v2tone_score(x):
    """
    Return the first numerical value of a string in an entry of V2Tone column from raw data file.
    :type: string
    :rtype: float
    """
    # l = x.split(',')
    return float(x.split(',')[0])

def clean_raw_data(data):
    news = data#[['DATE', 'DocumentIdentifier', 'V2Tone', 'Themes', 'Organizations', "V2Locations"]]
    # Drop missing value for org and themes; drop duplicate value for V2Tone(there are duplicate news which can affect our research performance)
    news_clean = news[~news['Organizations'].isna()]
    news_clean = news_clean[~news_clean['Themes'].isna()]

    # print('Cleaning corp...')
    index = news_clean['Organizations'].apply(lambda x: clean(x))
    news_clean['Organizations'] = index

    # print('Cleaning countries...')
    index = news_clean['Organizations'].apply(del_countries)
    news_clean['Organizations'] = index

    # print('Matching companies...')
    index = news_clean['Organizations'].apply(lambda x: company_matching(x, company_key1, company_dict))
    news_clean['Organizations'] = index
    # Drop missing values
    news_clean = news_clean.loc[index[~index.isnull()].index]
    # Drop duplicates based on tone,themes,organization
    news_clean['Organizations'] = news_clean['Organizations'].apply(lambda x: ','.join(x))
    news_clean = news_clean.drop_duplicates(subset=['V2Tone', 'Themes', 'Organizations'], keep='first')
    news_clean['Organizations'] = news_clean['Organizations'].apply(lambda x: x.split(','))
    # Return the first value in each entry from v2tone column
    news_clean['V2Tone'] = news_clean['V2Tone'].apply(lambda x: return_first_v2tone_score(x))

    return news_clean

def process_theme(news_clean):
    final_themes = []
    for i in range(len(news_clean['Themes'])):
        needed_themes = []
        row_themes = news_clean['Themes'].str.split(';').iloc[i]
        for j in row_themes:
            if j in themes:
                needed_themes.append(j)
        final_themes.append(needed_themes)

    news_clean['FinalThemes'] = final_themes
    news_clean['FinalThemes'] = [','.join(map(str, l)) for l in news_clean['FinalThemes']]
    news_clean['FinalThemes'] = news_clean['FinalThemes'].replace(r'^\s*$', np.nan, regex=True)
    news_clean = news_clean[news_clean['FinalThemes'].notna()]

    return news_clean

In [3]:
# Selecting companies, themes, nations
df = pd.read_pickle('company_dict_MXUS.pickle')
company_dict = {}
for key,value in df.items():
    key_temp=key.replace(',',' ')
    company_dict[key_temp.lower()] = value.lower()
company_key = list(company_dict.keys())
company_key1=','.join(company_key)
# print("company_key1",company_key)
df_theme = pd.read_pickle('theme_sdg_mapping.pk')
themes = list(df_theme.keys())

redundant = ['co', 'plc', 'ltd', '&', 'inc', 'company', 'corp', 'corporation']

nations = set(['afghanistan', 'albania', 'algeria', 'america', 'andorra', 'angola', 'antigua', 'argentina', 'armenia',
               'australia', 'austria', 'azerbaijan', 'bahamas', 'bahrain', 'bangladesh', 'barbados', 'belarus',
               'belgium', 'belize', 'benin', 'bhutan', 'bissau', 'bolivia', 'bosnia', 'botswana', 'brazil', 'british',
               'brunei', 'bulgaria', 'burkina', 'burma', 'burundi', 'cambodia', 'cameroon', 'canada', 'cape verde',
               'central african republic', 'chad', 'chile', 'china', 'colombia', 'comoros', 'congo', 'costa rica',
               'country debt', 'croatia', 'cuba', 'cyprus', 'czech', 'denmark', 'djibouti', 'dominica', 'east timor',
               'ecuador', 'egypt', 'el salvador', 'emirate', 'england', 'eritrea', 'estonia', 'ethiopia', 'fiji',
               'finland', 'france', 'gabon', 'gambia', 'georgia', 'germany', 'ghana', 'great britain', 'greece',
               'grenada', 'grenadines', 'guatemala', 'guinea', 'guyana', 'haiti', 'herzegovina', 'honduras', 'hungary',
               'iceland', 'in usa', 'india', 'indonesia', 'iran', 'iraq', 'ireland', 'israel', 'italy', 'ivory coast',
               'jamaica', 'japan', 'jordan', 'kazakhstan', 'kenya', 'kiribati', 'korea', 'kosovo', 'kuwait',
               'kyrgyzstan', 'laos', 'latvia', 'lebanon', 'lesotho', 'liberia', 'libya', 'liechtenstein', 'lithuania',
               'luxembourg', 'macedonia', 'madagascar', 'malawi', 'malaysia', 'maldives', 'mali', 'malta', 'marshall',
               'mauritania', 'mauritius', 'mexico', 'micronesia', 'moldova', 'monaco', 'mongolia', 'montenegro',
               'morocco', 'mozambique', 'myanmar', 'namibia', 'nauru', 'nepal', 'netherlands', 'new zealand',
               'nicaragua', 'niger', 'nigeria', 'norway', 'oman', 'pakistan', 'palau', 'panama', 'papua', 'paraguay',
               'peru', 'philippines', 'poland', 'portugal', 'qatar', 'romania', 'russia', 'rwanda', 'saint kitts',
               'samoa', 'san marino', 'santa lucia', 'sao tome', 'saudi arabia', 'scotland', 'scottish', 'senegal',
               'serbia', 'seychelles', 'sierra leone', 'singapore', 'slovakia', 'slovenia', 'solomon', 'somalia',
               'south africa', 'south sudan', 'spain', 'sri lanka', 'st kitts', 'st lucia', 'st. kitts', 'st. lucia',
               'sudan', 'suriname', 'swaziland', 'sweden', 'switzerland', 'syria', 'taiwan', 'tajikistan', 'tanzania',
               'thailand', 'tobago', 'togo', 'tonga', 'trinidad', 'tunisia', 'turkey', 'turkmenistan', 'tuvalu',
               'uganda', 'ukraine', 'united kingdom', 'united states', 'uruguay', 'usa', 'uzbekistan', 'vanuatu',
               'vatican', 'venezuela', 'vietnam', 'wales', 'welsh', 'yemen', 'zambia', 'zimbabwe'])


FileNotFoundError: [Errno 2] No such file or directory: 'company_dict_MXUS.pickle'

In [ ]:
# change 2 log file names, change url time span, change dir if needed

# Download the csv file
logging.basicConfig(filename='download_log_2024_Jan-May.txt', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
file_handler = logging.FileHandler('download_log_2024_Jan-May.txt')
file_handler.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
file_handler.setFormatter(formatter)
logging.getLogger('').addHandler(file_handler)
processed_urls = set()  # store processed URLs

lock = Lock()

def download_process(url):
    try:
        name = url.split("/")[-1]
        table_name = name.split(".")[0]

        start_time = time.time()  
        results, name, names = process_url(url)
        end_time = time.time()  
        download_time = end_time - start_time  
        with lock:
            logging.info(f"Download time for {name}: {download_time} seconds")

        data = clean_raw_data(results)
        final_all = process_theme(data)
        final_all.to_csv(f'/Volumes/T5EVO/gdelt_download/2024/{names[0]}.csv', index=True, header=True)

        with lock:
            log_message = f"Successfully downloaded from URL: {url} at {datetime.now()}"
            logging.info(log_message)
    except Exception as e:
        with lock:
            error_message = f"Error processing URL: {url} - {e}"
            logging.error(error_message)

def main():
    #urls = generate_gdelt_urls(2024,3,3, 2024,6,1)
    urls = ["http://data.gdeltproject.org/gdeltv2/20230921234500.gkg.csv.zip","http://data.gdeltproject.org/gdeltv2/20231010050000.gkg.csv.zip"]
    print("Total processing urls: ", len(urls))

    max_cores = psutil.cpu_count(logical=False)
    print(f"Using {max_cores} CPU cores for processing.")

    with ThreadPoolExecutor(max_workers=max_cores) as executor:
        executor.map(download_process, urls)

if __name__ == "__main__":
    main()

In [ ]:
# check missing table names

root_dir = '/Volumes/T5EVO/gdelt_download/2017'
schema_name = 'gdelt'
# Database connection URL
database_url = "postgresql://postgres@localhost:5432/gdelt"
try:
    engine = create_engine(database_url)

    with engine.connect() as connection:
        print("Connection successful.")
except Exception as e:
    print("Connection failed:", e)

# Get a list of all CSV files in the root directory
csv_files = [file[:-4] for file in os.listdir(root_dir) if file.endswith('.csv')]

# Initialize a list to store missing tables
missing_tables = []

# Check if each table exists in the database
for table_name in csv_files:
    sql = f"SELECT EXISTS (SELECT FROM information_schema.tables WHERE table_schema = 'gdelt' AND table_name = '{table_name}.csv');"
    result = engine.execute(sql)
    exists = result.scalar()
    if not exists:
        missing_tables.append(table_name + ".csv")

# Output missing tables
if missing_tables:
    print("\nTables not found in the database:")
    print(len(missing_tables))
else:
    print("\nAll tables are in the database.")

In [ ]:
# insert list with missing tables

root_dir = '/Volumes/T5EVO/gdelt_download/2017/'
database_url = "postgresql://postgres@localhost:5432/gdelt"

# Connect to the database
try:
    engine = create_engine(database_url)
    print("Database connection successful.")
except Exception as e:
    print("Database connection failed:", e)
    exit()

logging.basicConfig(filename='insertion_log_2017.txt', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# Remove any existing handlers
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

# Add a new file handler
file_handler = logging.FileHandler('insertion_log_2017.txt')
file_handler.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)


def copy_to_database(df, database_url, schema_name, table_name):
    try:
        start_time = time.time()
        table_name = f'"{schema_name}"."{table_name}.csv"'

        # Define column names and their corresponding data types
        column_types = {
            'GKGRECORDID': 'TEXT',
            'DATE': 'TEXT',
            'SourceCollectionIdentifier': 'TEXT',
            'SourceCommonName': 'TEXT',
            'DocumentIdentifier': 'TEXT',
            'Counts': 'TEXT',
            'V2Counts': 'TEXT',
            'Themes': 'TEXT',
            'V2Themes': 'TEXT',
            'Locations': 'TEXT',
            'V2Locations': 'TEXT',
            'Persons': 'TEXT',
            'V2Persons': 'TEXT',
            'Organizations': 'TEXT',
            'V2Organizations': 'TEXT',
            'V2Tone': 'FLOAT8', 
            'Dates': 'TEXT',
            'GCAM': 'TEXT',
            'SharingImage': 'TEXT',
            'RelatedImages': 'TEXT',
            'SocialImageEmbeds': 'TEXT',
            'SocialVideoEmbeds': 'TEXT',
            'Quotations': 'TEXT',
            'AllNames': 'TEXT',
            'Amounts': 'TEXT',
            'TranslationInfo': 'TEXT',
            'Extras': 'TEXT',
            'FinalThemes': 'TEXT'
        }

        # Generate the CREATE TABLE query based on column names and types
        create_table_sql = f"CREATE TABLE IF NOT EXISTS {table_name} (\n"
        for col, col_type in column_types.items():
            create_table_sql += f"{col} {col_type},\n"
        create_table_sql = create_table_sql.rstrip(',\n') + "\n)"

        with engine.connect() as conn:
            with conn.begin():
                conn.execute(create_table_sql)
                

                copy_sql = f"""
                    COPY {table_name}
                    FROM STDIN
                    WITH CSV HEADER DELIMITER AS '\t'
                """

                # Write DataFrame to a buffer
                buffer = StringIO()
                df.to_csv(buffer, sep='\t', header=True, index=False, quoting=csv.QUOTE_NONNUMERIC)
                buffer.seek(0)
                print(f"DataFrame written to buffer for {table_name}")

                # Copy data from buffer to database
                conn.connection.cursor().copy_expert(copy_sql, buffer)
                print(f"Data copied to table {table_name}")

        end_time = time.time()
        insertion_time = end_time - start_time
        logging.info(f"Insertion time for {table_name}: {insertion_time} seconds")

    except Exception as e:
        error_message = f"Error copying data to database for table {table_name}: {e}"
        logging.error(error_message)
        print("copy_to_database error: ", error_message)
        raise  # Re-raise the exception to identify the specific line of the error

def process_csv(file_path):
    try:
        print(f"Starting process_csv for file: {file_path}")
        name = os.path.basename(file_path)
        year = name.split(".")[0][:4]


        table_name = name.split(".")[0]

        inspector = inspect(engine)
        schema_name = "gdelt"
        if inspector.has_table(table_name, schema=schema_name):
            logging.info(f"Table {table_name} skipped because it exists.")
            return

        # Read the CSV file using pandas within a context manager
        df = pd.read_csv(file_path).drop(["Unnamed: 0"], axis=1)
        print(f"CSV file {file_path} read into DataFrame")

        # Process the CSV file and insert data into the database
        copy_to_database(df, database_url, schema_name, table_name)

        log_message = f"Successfully wrote table from file: {file_path} at {datetime.now()}"
        logging.info(log_message)

    except Exception as e:
        error_message = f"Error processing file: {file_path} - {e}"
        logging.error(error_message)
        print("process_csv error: ", error_message)
        raise  # Re-raise the exception to identify the specific line of the error
        
# Create full file paths by joining the root directory and file names
full_file_paths = [f"{root_dir}{file_name}" for file_name in missing_tables]
print("len of processing csv: ",len(full_file_paths))

# Insert CSV files not found in the database
if full_file_paths:
    for file_path in full_file_paths:
        try:
            process_csv(file_path)
        except Exception as e:
            print(f"Error processing file: {file_path} - {e}")
else:
    print("All CSV files are already in the database.")

The following is to change all files name ending with ".csv" as required

In [ ]:
# change table name (add '.csv')

database_url = "postgresql://postgres@localhost:5432/gdelt"
try:
    conn = psycopg2.connect(database_url)
    print("Database connection successful.")
except Exception as e:
    print("Database connection failed:", e)
    exit()
    
cur = conn.cursor()

# Query to get table names that do not end with '.csv'
query = """
SELECT table_name 
FROM information_schema.tables 
WHERE table_schema = 'gdelt' AND table_name NOT LIKE '%.csv';
"""

try:
    # Execute the query
    cur.execute(query)

    # Fetch all the table names
    table_names = cur.fetchall()

    if table_names:
        print("Tables that do not end with '.csv':")
        for table in table_names:
            print(table[0])
            
        # Rename tables
        for table in table_names:
            old_table_name = table[0]
            new_table_name = old_table_name + '.csv'
            rename_query = sql.SQL("ALTER TABLE {}.{} RENAME TO {}").format(
                sql.Identifier('gdelt'),
                sql.Identifier(old_table_name),
                sql.Identifier(new_table_name)
            )
            cur.execute(rename_query)
            print(f"Renamed {old_table_name} to {new_table_name}")
            
        # Commit the changes
        conn.commit()
    else:
        print("No tables found that do not end with '.csv'")

except Exception as e:
    print("Error executing query or renaming tables:", e)
    conn.rollback()
finally:
    cur.close()
    conn.close()